<a href="https://colab.research.google.com/github/JasperLS/Understanding_Videos_at_Scale/blob/master/scraping/ufc_stats_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrape Data


### General Prep

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time as time_lib
import os
import csv
import pandas as pd
import sys
chrome_options = Options()
chrome_options.add_argument("--headless")
import datetime

browser = webdriver.Chrome("D:/shared/chromedriver79.exe", options=chrome_options)
date =str(datetime.datetime.today().strftime('%Y-%m-%d'))
print(date)

2020-03-16


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"#"last_expr" #all for debugging better

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings("ignore")

### Get All Stats

In [ ]:
# define input
alphabet = 'A B C D E F G H I J K L M N O P Q R S T U V W X Y Z'.split(' ')
FID = 0

cols_history = ['Result','Fighter_1','Fighter_2','STR_1','STR_2','TD_1','TD_2','SUB_1','SUB_2','PASS_1','PASS_2','Event','Date','Method','Round','Time']
cols_match = ['MID','FID','Result_F1','Event','Date','Method','Round','Time',                                         # before click
              'Type','T_Format','Ref','Details',                                                             # after click
              'Fighter_1','Fighter_2',
              'KD_1','KD_2','SigStr_e_1','SigStr_t_1','SigStr_e_2','SigStr_t_2','TotStr_e_1','TotStr_t_1','TotStr_e_2','TotStr_t_2','TD_e_1','TD_t_1','TD_e_2','TD_t_2',
              'SUB_1','SUB_2','PASS_1','PASS_2','REV_1','REV_2',
              'Head_e_1','Head_t_1','Head_e_2','Head_t_2','Body_e_1','Body_t_1','Body_e_2','Body_t_2','Leg_e_1','Leg_t_1','Leg_e_2','Leg_t_2',
              'Distance_e_1','Distance_t_1','Distance_e_2','Distance_t_2','Clinch_e_1','Clinch_t_1','Clinch_e_2','Clinch_t_2','Ground_e_1','Ground_t_1','Ground_e_2','Ground_t_2']
cols_plan = ['FID','Fighter_1','Fighter_2']
cols_all_fighters = ['FID','First','Last','Nick','HT','WT','Reach','Stance','W','L','D']
cols_profile = ['FID','HT','WT','Reach','Stance','DOB']

plan = pd.DataFrame(columns = cols_plan)
plan_path = f'D:\\jasper\\98_general_projects\\03_UFC_Predictions\\Data\\{date}_Plan_List.csv'
plan.to_csv(plan_path, mode = 'w',header = True, index=False)

matches = pd.DataFrame(columns = cols_match)
matches_path = f'D:\\jasper\\98_general_projects\\03_UFC_Predictions\\Data\\{date}_Matches_List.csv'
matches.to_csv(matches_path, mode = 'w',header = True, index=False)

fighters = pd.DataFrame(columns = cols_all_fighters)
fighters_path = f'D:\\jasper\\98_general_projects\\03_UFC_Predictions\\Data\\{date}_Fighters_List.csv'
fighters.to_csv(fighters_path, mode = 'w',header = True,index=False)

profile = pd.DataFrame(columns = cols_profile)
profile_path = f'D:\\jasper\\98_general_projects\\03_UFC_Predictions\\Data\\{date}_Profiles_List.csv'
profile.to_csv(profile_path, mode = 'w',header = True,index=False)

In [ ]:
def match_page(browser,Event,Date,Method,Roundn,Time,FID,cols_match):
    MID = browser.current_url.split('/')[-1]
    Result = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/i').text
    Type = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[1]/i').text
    T_Format = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[2]/p[1]/i[4]').text.replace('TIME FORMAT: ','')
    try: Ref = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[2]/p[1]/i[5]/span').text
    except: Ref = None
    Details = browser.find_element_by_xpath(f'/html/body/section/div/div/div[2]/div[2]/p[2]').text.replace('DETAILS: ','')
    
    try:    Fighter_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/div/h3/a').text
    except: Fighter_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/div/h3/span').text
    try:    Fighter_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[2]/div/h3/a').text
    except: Fighter_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[2]/div/h3/span').text
    
    try: # get detailed significant strike infos
        KD_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[1]').text
        KD_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[2]/p[2]').text

        SigStr_e_1,SigStr_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[1]').text.split(' of ')
        SigStr_e_2,SigStr_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[3]/p[2]').text.split(' of ')

        TotStr_e_1,TotStr_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[1]').text.split(' of ')
        TotStr_e_2,TotStr_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[5]/p[2]').text.split(' of ')

        TD_e_1,TD_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[1]').text.split(' of ')
        TD_e_2,TD_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[6]/p[2]').text.split(' of ')

        SUB_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[1]').text
        SUB_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[8]/p[2]').text
        PASS_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[1]').text
        PASS_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[9]/p[2]').text
        REV_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[1]').text
        REV_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/section[2]/table/tbody/tr/td[10]/p[2]').text

        Head_e_1,Head_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[4]/p[1]').text.split(' of ')
        Head_e_2,Head_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[4]/p[2]').text.split(' of ')

        Body_e_1,Body_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[5]/p[1]').text.split(' of ')
        Body_e_2,Body_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[5]/p[2]').text.split(' of ')

        Leg_e_1,Leg_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[6]/p[1]').text.split(' of ')
        Leg_e_2,Leg_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[6]/p[2]').text.split(' of ')

        Distance_e_1,Distance_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[7]/p[1]').text.split(' of ')
        Distance_e_2,Distance_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[7]/p[2]').text.split(' of ')

        Clinch_e_1,Clinch_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[8]/p[1]').text.split(' of ')
        Clinch_e_2,Clinch_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[8]/p[2]').text.split(' of ')

        Ground_e_1,Ground_t_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[9]/p[1]').text.split(' of ')
        Ground_e_2,Ground_t_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr/td[9]/p[2]').text.split(' of ')

    except:
        KD_1,KD_2,SigStr_e_1,SigStr_t_1,SigStr_e_2,SigStr_t_2,TotStr_e_1,TotStr_t_1,TotStr_e_2,TotStr_t_2,TD_e_1,TD_t_1,TD_e_2,TD_t_2,\
                                      SUB_1,SUB_2,PASS_1,PASS_2,REV_1,REV_2,\
                                      Head_e_1,Head_t_1,Head_e_2,Head_t_2,Body_e_1,Body_t_1,Body_e_2,Body_t_2,Leg_e_1,Leg_t_1,Leg_e_2,Leg_t_2,\
                                      Distance_e_1,Distance_t_1,Distance_e_2,Distance_t_2,\
                                      Clinch_e_1,Clinch_t_1,Clinch_e_2,Clinch_t_2,Ground_e_1,Ground_t_1,Ground_e_2,Ground_t_2 = [None]*44
        pass
    
    return pd.DataFrame([[MID,FID,Result,Event,Date,Method,Roundn,Time,
                                          Type,T_Format,Ref,Details,
                                          Fighter_1,Fighter_2,
                                          KD_1,KD_2,SigStr_e_1,SigStr_t_1,SigStr_e_2,SigStr_t_2,TotStr_e_1,TotStr_t_1,TotStr_e_2,TotStr_t_2,TD_e_1,TD_t_1,TD_e_2,TD_t_2,
                                          SUB_1,SUB_2,PASS_1,PASS_2,REV_1,REV_2,
                                          Head_e_1,Head_t_1,Head_e_2,Head_t_2,Body_e_1,Body_t_1,Body_e_2,Body_t_2,Leg_e_1,Leg_t_1,Leg_e_2,Leg_t_2,
                                          Distance_e_1,Distance_t_1,Distance_e_2,Distance_t_2,
                                          Clinch_e_1,Clinch_t_1,Clinch_e_2,Clinch_t_2,Ground_e_1,Ground_t_1,Ground_e_2,Ground_t_2]], columns = cols_match)   

In [ ]:
def fighter_page(browser,j):
    Event = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[7]/p[1]/a').text
    Date = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[7]/p[2]').text
    Method = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[8]/p[1]').text
    Roundn = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[9]/p').text
    Time = browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[10]/p').text
    
    return Event,Date,Method,Roundn,Time

In [ ]:
def fighter_profile(browser,FID,cols_profile):
    height = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[1]').text.replace('HEIGHT: ','')
    weight = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[2]').text.replace('WEIGHT: ','')
    reach = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[3]').text.replace('REACH: ','')
    stance = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[4]').text.replace('STANCE: ','')
    dob = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/ul/li[5]').text.replace('DOB: ','')
    
    profile = pd.DataFrame([[int(FID),height,weight,reach,stance,dob]],columns = cols_profile)
    return profile

In [ ]:
def plan_page(browser, FID,cols_plan):
    try:
        Fighter_1 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[1]/div/h3/a').text
        Fighter_2 = browser.find_element_by_xpath(f'/html/body/section/div/div/div[1]/div[2]/div/h3/a').text
    except:
        Fighter_1,Fighter_2 = 'Error','Error'
        
    return pd.DataFrame([[FID,Fighter_1,Fighter_2]], columns = cols_plan) 

In [ ]:
def all_fighters_page(browser,i, FID, cols_all_fighters):
            First = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[1]/a').text
            Last = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[2]/a').text
            Nick = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[3]/a').text
            HT = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[4]').text
            WT = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[5]').text
            Reach = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[6]').text
            Stance = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[7]').text
            W = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[8]').text
            L = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[9]').text
            D = browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[10]').text

            fighters = pd.DataFrame([[FID,First, Last, Nick, HT, WT, Reach, Stance, W, L, D]],columns = cols_all_fighters)

            return fighters

In [ ]:
# total loop with all infor included
from itertools import count
time_lib.ctime()
FID

for letter in alphabet: # loop through starting letters A to Z
    website = f'http://ufcstats.com/statistics/fighters?char={letter.lower()}&page=all'
    browser.get(website)
    
    for i in count(2): # loop through list of fighters on page with starting letter
        try: # get fighter stats
            fighters = all_fighters_page(browser,i,FID,cols_all_fighters)
            fighters.to_csv(fighters_path, mode = 'a+',header = False, index = False)
        except: pass
        
        try:
            browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[1]/a').click()#;time.sleep(3)
            try: 
                profile = fighter_profile(browser,FID,cols_profile)
                profile.to_csv(profile_path, mode = 'a+', header = False, index = False)
            except: pass
            
            for j in count(2): # loop through list of fights per fighter
                try:
                    if browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').text == 'NEXT':
                        browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').click()#;time_lib.sleep(3) # go to plan page
                        plan = plan_page(browser, FID,cols_plan)
                        plan.to_csv(plan_path, mode = 'a+',header = False, index = False)
                        browser.execute_script("window.history.go(-1)");time_lib.sleep(0.5) # go back to fighter page
                    
                    else:
                        Event,Date,Method,Roundn,Time = fighter_page(browser, j)

                        browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').click()#;time_lib.sleep(3) # go to match page
                        matches = match_page(browser,Event,Date,Method,Roundn,Time,FID,cols_match)
                        matches.to_csv(matches_path, mode = 'a+',header = False, index = False)
                        browser.execute_script("window.history.go(-1)");time_lib.sleep(0.5) # go back to fighter page
                
                except: # end loop trough list of fights
                    print(FID, 'completed with j:',j)
                    FID += 1
                    browser.execute_script("window.history.go(-1)");time_lib.sleep(0.5) # go back to fighter list page            
                    break
                    
        except: # end loop through list of fighers per starting letter
            print(letter,'max i: ',i,' and time:',time_lib.ctime())
            break

In [ ]:
# total loop with all infor included
from itertools import count
time_lib.ctime()
FID = 3364

for letter in ['t']: # loop through starting letters A to Z
    website = f'http://ufcstats.com/statistics/fighters?char={letter.lower()}&page=all'
    browser.get(website)
    
    for i in count(80): # loop through list of fighters on page with starting letter
        try: # get fighter stats
            fighters = all_fighters_page(browser,i,FID,cols_all_fighters)
            fighters.to_csv(fighters_path, mode = 'a+',header = False, index = False)
        except: pass
        
        try:
            browser.find_element_by_xpath(f'/html/body/section/div/div/div/table/tbody/tr[{i}]/td[1]/a').click()#;time.sleep(3)
            try: 
                profile = fighter_profile(browser,FID,cols_profile)
                profile.to_csv(profile_path, mode = 'a+', header = False, index = False)
            except: pass
            
            for j in count(2): # loop through list of fights per fighter
                try:
                    if browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').text == 'NEXT':
                        browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').click()#;time_lib.sleep(3) # go to plan page
                        plan = plan_page(browser, FID,cols_plan)
                        plan.to_csv(plan_path, mode = 'a+',header = False, index = False)
                        browser.execute_script("window.history.go(-1)");time_lib.sleep(0.5) # go back to fighter page
                    
                    else:
                        Event,Date,Method,Roundn,Time = fighter_page(browser, j)

                        browser.find_element_by_xpath(f'/html/body/section/div/div/table/tbody/tr[{j}]/td[1]/p/a/i/i').click()#;time_lib.sleep(3) # go to match page
                        matches = match_page(browser,Event,Date,Method,Roundn,Time,FID,cols_match)
                        matches.to_csv(matches_path, mode = 'a+',header = False, index = False)
                        browser.execute_script("window.history.go(-1)");time_lib.sleep(0.5) # go back to fighter page
                
                except: # end loop trough list of fights
                    print(FID, 'completed with j:',j)
                    FID += 1
                    browser.execute_script("window.history.go(-1)");time_lib.sleep(0.5) # go back to fighter list page            
                    break
                    
        except: # end loop through list of fighers per starting letter
            print(letter,'max i: ',i,' and time:',time_lib.ctime())
            break

'Tue Mar 17 20:52:07 2020'

3364

3364 completed with j: 18
3365 completed with j: 2
3366 completed with j: 6
3367 completed with j: 6
3368 completed with j: 3
3369 completed with j: 3
3370 completed with j: 3
3371 completed with j: 4
3372 completed with j: 14
3373 completed with j: 4
3374 completed with j: 3
3375 completed with j: 13
3376 completed with j: 3
3377 completed with j: 2
3378 completed with j: 4
3379 completed with j: 2
3380 completed with j: 5
3381 completed with j: 5
3382 completed with j: 3
3383 completed with j: 4
3384 completed with j: 4
3385 completed with j: 6
3386 completed with j: 5
3387 completed with j: 2
3388 completed with j: 15
3389 completed with j: 23
3390 completed with j: 2
3391 completed with j: 5
3392 completed with j: 2
3393 completed with j: 6
3394 completed with j: 3
3395 completed with j: 3
3396 completed with j: 13
3397 completed with j: 4
3398 completed with j: 6
3399 completed with j: 11
3400 completed with j: 6
3401 completed with j: 5
3402 completed with j: 3
3403 completed wit

## Save Model

In [ ]:
import os
import subprocess
import datetime

In [ ]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [ ]:
# Optional: Save Dataframe
date = datetime.datetime.today().strftime('%Y-%m-%d')
# try: df.to_csv(f'D:/jasper/03_stoyo/04_Data_Stoyo_2/CSV only videos/combined/{date}_all_clients_combined.csv')
# except: pass

In [ ]:
%notebook "D:/jasper/98_general_projects/03_UFC_Predictions/Backup/ufcstats_scraper.ipynb"  
_ = subprocess.call(f'jupyter nbconvert ufcstats_scraper.ipynb --output D:\\jasper\\98_general_projects\\03_UFC_Predictions/Backup/{date}_ufcstats_scraper.html"'); del _
print('\nFEIERABEND!!!')


FEIERABEND!!!
